# Outbound Auth

Outbound Auth는 Agent와 AgentCore Gateway가 Inbound Auth 중에 인증 및 권한이 부여된 사용자를 대신하여 AWS 리소스 및 타사 서비스에 안전하게 액세스할 수 있도록 합니다. AWS 리소스  또는  타사 서비스와 권한 부여를 통합하려면 Inbound Auth와 Outbound Auth를 모두 구성해야 합니다.

AgentCore Identity가 지원하는 최소 권한 액세스 및 안전한 권한 위임을 통해 Agent는 GitHub, Google, Salesfor ce, Slack과 같은 AWS 리소스 및 타사 Tool에 원활하고 안전하게 액세스할 수 있습니다. Agent는 사전 승인된 사용자 동의가 있는 경우 사용자를 대신하거나 독립적으로 이러한 서비스에서 작업을 수행할 수 있습니다. 또한 안전한 토큰 볼트를 사용하여 동의 피로를 줄이고 간소화된 AI Agent 경험을 만들 수 있습니다.

## Outbound Authentication 구성

먼저 타사 Provider에 클라이언트 애플리케이션을 등록한 다음 Outbound Auth를 생성합니다. AWS 리소스  또는  타사 서비스  또는  AgentCore Gateway 대상에 대한 액세스를 검증하는 방법을 지정합니다. OAuth 2LO/3LO  또는  API 키를 사용할 수 있습니다. OAuth를 사용하면 AgentCore Identity가 제공하는 Provider 중에서 선택할 수 있습니다. 이 경우 AgentCore Identity에서 Provider에 대한 구성 세부 정보를 입력합니다.  또는  커스텀 Provider에 대한 세부 정보를 제공할 수 있습니다.

사용자가 AWS 리소스  또는  타사 서비스  또는  AgentCore Gateway 대상에 액세스하려는 경우 Outbound Auth는 Incoming Auth에서 제공한 액세스 토큰이 유효한지 확인하고 유효한 경우 리소스에 대한 액세스를 허용합니다.

<div style="text-align:center">
    <img src="images/outbound_auth.png" width="90%"/>
</div>

## Resource credential provider

이것은 Agent 코드가 다운스트림 리소스 서버(e.g. Google, GitHub)의 자격 증명을 검색하여 액세스하는 데 사용하는 구성 요소입니다(e.g. Gmail에서 이메일 가져오기, Google Calendar에 회의 추가). Agent 개발자가 최종 사용자, Agent 코드 및 외부 권한 부여 서버 간에 2LO 및 3LO OAuth2 오케스트레이션 플로우를 구현하는 부담을 덜어줍니다. AgentCore는 커스텀 OAuth2 credential provider와 Google, GitHub, Slack, Salesfor ce와 같은 내장 Provider 목록을 제공하며 권한 부여 서버 엔드포인트 및 Provider별 매개변수가 미리 채워져 있습니다.
  

Bedrock AgentCore Identity는 Agent 개발자가 OAuth2  또는  API 키를 지원하는 외부 리소스로 인증할 수 있도록 OAuth2 및 API Key Credential Provider를 제공합니다. 다음 예제에서는 API Key credential provider를 구성하는 방법을 안내합니다. 그런 다음 Agent는 API Key credential provider를 사용하여 모든 Agent 작업에 대한 API 키를 검색할 수 있습니다. 다른 credential provider에 대해서는 문서를 참조하십시오.

 ### resource credential provider 생성

 다음은 API Key resource credential provider를 생성하는 예제입니다.

```
from bedrock_agentcore.services.identity import IdentityClient
identity_client = IdentityClient(region="us-west-2")

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "APIKey-provider-name",
    "apiKey": "<my-api-key>" # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
})
print(api_key_provider)
```

 ### Resource credential provider에서 액세스 토큰  또는  API 키 검색

다음은 API Key credential provider에서 API 키를 검색하는 예제입니다. Agent는 API 키를 사용하여 LLM과 같은 서비스  또는  API 키 구성을 사용하는 다른 서비스와 상호 작용할 수 있습니다. credential provider에서 access_token  또는  API 키와 같은 자격 증명을 검색하려면 아래와 같이 함수를 데코레이트할 수 있습니다.

```
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key

@requires_api_key(
    provider_name="APIKey-provider" # replace with your own credential provider name
)
async def need_api_key(*, api_key: str):
    print(f'received api key for  async func: {api_key}')

await need_api_key(api_key="")
```

다음은 @require_access_token 데코레이터와 함께 사용할 수 있는 다양한 매개변수입니다.


| Parameter Name      | Description                                                              |
|:--------------------|:-------------------------------------------------------------------------|
| provider_name       | credential provider 이름                                             |
| into                | 토큰을 주입할 매개변수 이름                                  |
| scopes              | 요청할 OAuth2 범위                                                 |
| on_auth_url	      | 권한 부여 URL 처리를 위한 콜백                                 |
| auth_flow           | 인증 플로우 유형 ("M2M"  또는  "USER_FEDERATION")                    |
| callback_url        | OAuth2 콜백 URL                                                      |
| for ce_authentication| 재인증 강제                                                  |
| token_poller        | 커스텀 토큰 폴러 구현                                       |

		


# Amazon Bedrock AgentCore Runtime에서 OpenAI Model을 사용하는 Strands Agent 호스팅

## 개요


이 튜토리얼에서는 01-AgentCore-runtime에서 배포한 openai-model을 사용하는 Agent를 수정하고 API Key credential provider를 사용하여 Outbound Auth를 구성합니다. API Key credential provider를 설정하여 open-ai 키를 저장하고 이 키를 사용하도록 Agent 코드를 수정합니다.

### 튜토리얼 아키텍처

<div style="text-align:center">
    <img src="images/outbound_auth_api.png" width="90%"/>
</div>


### 튜토리얼 세부 정보

| Infor mation         | Details                                                                  |
|:--------------------|:-------------------------------------------------------------------------|
| 튜토리얼 유형       | Conversational                                                           |
| Agent 유형          | Single                                                                   |
| Agentic Framework   | Strands Agents                                                           |
| LLM model           | GPT 4.1 mini                                                             |
| 튜토리얼 구성 요소 | AgentCore Runtime에서 Agent 호스팅. Strands Agent 및 OpenAI Model 사용 |
| 튜토리얼 분야   | Cross-vertical                                                           |
| 예제 복잡도  | Easy                                                                     |
| 사용된 SDK            | Amazon BedrockAgentCore Python SDK and boto3                             |
| Credential Provider | Type : API Key                                                           |


### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore Runtime에서 Agent 호스팅
* OpenAI Model 사용
* Strands Agent 사용
* API Key credential provider와 함께 AgentCore egress Auth 사용


## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker 실행 중
* OpenAI API Keys

OpenAI API 키를 얻는 방법:
- OpenAI: [OpenAI API keys](https://help.openai.com/en/articles/4936850-where-do-i-find-my-openai-api-key)
- Azure OpenAI: [Create Azure OpenAI resource and get keys](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?tabs=azure-portal)

실행 전 환경 변수 설정:
- OpenAI:
  - `OPENAI_API_KEY`
- Azure OpenAI:
  - `AZURE_OPENAI_API_KEY`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_API_VERSION` (e.g., `2024-02-15-preview`)
  - `AZURE_OPENAI_DEPLOYMENT` (배포된 모델 이름)

선택적 Provider 전환:
- `OPENAI_PROVIDER` = `openai` (기본값)  또는  `azure`

참고:
- Do not hardcode secrets. Use AgentCore Identity’s credential provider to store and retrieve API keys at runtime, and rotate keys regularly.

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Agent 생성 및 로컬 실험

AgentCore Runtime에 Agent를 배포하기 전에 실험 목적으로 로컬에서 개발하고 실행해 보겠습니다.

프로덕션 에이전트 애플리케이션의 경우 Agent 생성 프로세스를 Agent 호출 프로세스와 분리해야 합니다. AgentCore Runtime을 사용하면 Agent의 호출 부분을 `@app.entrypoint` 데코레이터로 장식하고 이를 런타임의 진입점으로 사용합니다. 먼저 실험 단계에서 각 Agent가 어떻게 개발되는지 살펴보겠습니다.

여기서 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator  # Strands 프레임워크의 계산기 도구
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os

# Azure OpenAI API 설정 (실제 값으로 교체 필요)
os.environ["AZURE_API_KEY"] = "<YOUR_API_KEY>"
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# 커스텀 날씨 도구 정의
@tool
def weather():
    """ Get weather """
    return "sunny"  # 더미 구현

model = "azure/gpt-4.1-mini"
# LiteLLM을 통해 Azure OpenAI 모델 래핑
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
)

# Agent 초기화: 모델과 도구들을 연결
agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    # 응답 메시지에서 텍스트 추출
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    # JSON 문자열을 파싱하여 agent 호출
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

#### 로컬 Agent 호출

In [ ]:
!python strands_agents_openai.py '{"prompt": "What is the weather now?"}'

## Resource Credential Provider 생성

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

from boto3.session import Session
import boto3

boto_session = Session()
region = boto_session.region_name

# API Key Credential Provider 설정
identity_client = IdentityClient(region=region)

# API Key를 안전하게 저장하는 credential provider 생성
api_key_provider = identity_client.create_api_key_credential_provider(
    {
        "name": "openai-apikey-provider",
        "apiKey": "<YOUR_API_KEY>",  # OpenAI 등 외부 서비스에서 발급받은 API 키로 교체
    }
)
print(api_key_provider)

## AgentCore Runtime에 배포하기 위한 Agent 준비 및 Resource Credential Provider 사용

이제 AgentCore Runtime에 Agent를 배포해 보겠습니다. 이를 위해 다음이 필요합니다:
* Imp 또는 t the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Dec 또는 ate the invocation function with the `@app.entrypoint` dec 또는 at 또는 
* Let AgentCoreRuntime control the running of the agent with `app.run()`
* Retrieve the openAI key from the resource credential provider created in the previous step

### OpenAI Model을 사용하는 Strands Agent
GPT 4.1 mini Model을 사용하는 Strands Agent부터 시작하겠습니다. 다른 모든 것도 정확히 동일하게 작동합니다.

In [ ]:
%%writefile strands_agents_openai.py
import asyncio
from bedrock_agentcore.identity.auth import requires_access_token, requires_api_key
from strands import Agent, tool
from strands_tools import calculator 
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Credential Provider에서 가져온 API 키를 저장할 전역 변수
AZURE_API_KEY_FROM_CREDS_PROVIDER = ""


@requires_api_key(
    provider_name="openai-apikey-provider"  # 이전에 생성한 credential provider 이름
)
async def need_api_key(*, api_key: str):
    global AZURE_API_KEY_FROM_CREDS_PROVIDER
    print(f'received api key for async func: {api_key}')
    AZURE_API_KEY_FROM_CREDS_PROVIDER = api_key

# 모듈 레벨에서는 빈 값 출력하지 않음 - entrypoint 함수에서 출력

# AgentCore Runtime 앱 초기화
app = BedrockAgentCoreApp()

# API 키는 entrypoint 함수에서 동적으로 설정됨
# Azure OpenAI 설정 (API 키 제외)
os.environ["AZURE_API_BASE"] = "<YOUR_API_BASE>"
os.environ["AZURE_API_VERSION"] = "<YOUR_API_VERSION>"

# 커스텀 날씨 도구 정의
@tool
def weather():
    """ Get weather """
    return "sunny"  # 더미 구현

# Agent는 API 키 설정 후 초기화되므로 전역 변수로 선언
agent = None

@app.entrypoint
async def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    global AZURE_API_KEY_FROM_CREDS_PROVIDER, agent
    
    print(f"Entrypoint called with AZURE_API_KEY_FROM_CREDS_PROVIDER: '{AZURE_API_KEY_FROM_CREDS_PROVIDER}'")
    
    # API 키가 아직 없으면 credential provider에서 가져오기
    if not AZURE_API_KEY_FROM_CREDS_PROVIDER:
        print("Attempting to retrieve API key...")
        try:
            await need_api_key(api_key="")
            print(f"API key retrieved: '{AZURE_API_KEY_FROM_CREDS_PROVIDER}'")
            # 환경 변수에 API 키 설정
            os.environ["AZURE_API_KEY"] = AZURE_API_KEY_FROM_CREDS_PROVIDER
            print("Environment variable AZURE_API_KEY set")
        except Exception as e:
            print(f"Error retrieving API key: {e}")
            raise
    else:
        print("API key already available")
    
    # API 키 설정 후 Agent 초기화 (한 번만 실행)
    if agent is None:
        print("Initializing agent with API key...")
        model = "azure/gpt-4.1-mini"
        # LiteLLM을 통해 Azure OpenAI 모델 래핑
        litellm_model = LiteLLMModel(
            model_id=model, params={"max_tokens": 32000, "temperature": 0.7}
        )
        
        # Agent 초기화: 모델과 도구들을 연결
        agent = Agent(
            model=litellm_model,
            tools=[calculator, weather],
            system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
        )
        print("Agent initialized successfully")
    
    user_input = payload.get("prompt")
    print(f"User input: {user_input}")
    
    try:
        response = agent(user_input)
        print(f"Agent response: {response}")
        # 응답 메시지에서 텍스트 추출
        return response.message['content'][0]['text']
    except Exception as e:
        print(f"Error in agent processing: {e}")
        raise

if __name__ == "__main__":
    app.run()


## 내부적으로 무슨 일이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로:

* Creates an HTTP server that listens on the p 또는 t 8080
* Implements the required `/invocations` endpoint for  processing the agent's requirements
* Implements the `/ping` endpoint for  health checks (very important for  asynchronous agents)
* Handles proper content types and response for mats
* Manages err 또는  handling acc 또는 ding to the AWS standards

## AgentCore Runtime에 Agent 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 Agent와 통신하는 방법을 제어할 수 있습니다. 

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다

이 튜토리얼에서는 Amazon Bedrock AgentCode Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore Runtime에 배포합니다.

### AgentCore Runtime 배포 구성

다음으로 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 요구 사항 파일로 AgentCore Runtime 배포를 구성합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json

boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()
agent_name = "strands_agents_openai"

# AgentCore Runtime 배포 구성
# - 실행 역할과 ECR 리포지토리 자동 생성
# - Dockerfile 자동 생성
response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    region=region,
)
response

### AgentCore Runtime에 Agent 시작

이제 docker 파일이 있으므로 AgentCore Runtime에 Agent를 시작하겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 AgentCore Runtime이 생성됩니다

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()
launch_result

#### 자동 생성된 역할에 추가 필수 정책 추가

Agent에 일부 아웃바운드 ID를 추가하고 있으므로 자동 생성된 역할에서 사용할 수 없는 일부 API 키 및 비밀을 가져와야 합니다. 이를 위해 자동 생성된 IAM 역할에 몇 가지 추가 권한을 추가해야 합니다. 먼저 이 역할을 가져온 다음 해당 권한을 추가하겠습니다.

In [ ]:
import json

agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

# 배포된 Runtime 정보에서 실행 역할 ARN 가져오기
runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response["roleArn"]

# Outbound Auth를 위한 추가 권한 정의
# - API Key 조회 권한
# - Secrets Manager 접근 권한
policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": ["bedrock-agentcore:GetResourceApiKey"],
            "Resource": "*",
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": ["secretsmanager:GetSecretValue"],
            "Resource": "arn:aws:secretsmanager:*:*:secret:bedrock-agentcore*",
        },
    ],
}
iam_client = boto3.client("iam", region_name=region)

# IAM 역할에 정책 추가
response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인하겠습니다

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
# Runtime이 준비 상태가 될 때까지 폴링
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]
    print(status)
status

### AgentCore Runtime 호출

마지막으로 페이로드를 사용하여 AgentCore Runtime을 호출할 수 있습니다

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke(
    {"prompt": "Hello"}, user_id="userid_1234567890"  # user_id는 추적 및 권한 관리에 사용
)
invoke_response

### 호출 결과 처리

이제 호출 결과를 처리하여 애플리케이션에 포함할 수 있습니다

In [ ]:
from IPython.display import Markdown, display

# 응답에서 텍스트 추출하여 Markdown으로 표시
response_text = invoke_response["response"][0]
display(Markdown(response_text))

### AgentCore Runtime 호출 with boto3

이제 AgentCore Runtime이 생성되었으므로 모든 AWS SDK로 호출할 수 있습니다. 예를 들어 boto3 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client("bedrock-agentcore", region_name=region)

# boto3를 사용한 직접 호출 (SDK 대신)
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    runtimeUserId="userid_1234567890",
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"}),
)
# 응답 타입에 따라 다르게 처리 (스트리밍 vs 일반)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## 정리 (선택 사항)

이제 생성된 AgentCore Runtime을 정리하겠습니다

In [ ]:
agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
ecr_client = boto3.client("ecr", region_name=region)

iam_client = boto3.client("iam")

# AgentCore Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

# ECR 리포지토리 삭제 (이미지 포함)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split("/")[1], force=True
)

# IAM 역할에 연결된 모든 정책 삭제
policies = iam_client.list_role_policies(
    RoleName=runtime_role.split("/")[1], MaxItems=100
)

for policy_name in policies["PolicyNames"]:
    iam_client.delete_role_policy(
        RoleName=runtime_role.split("/")[1], PolicyName=policy_name
    )
# IAM 역할 삭제
iam_response = iam_client.delete_role(RoleName=runtime_role.split("/")[1])

# 축하합니다!